In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime as dt
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import tree
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.metrics import r2_score
import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout
from sklearn.preprocessing import StandardScaler,MinMaxScaler
import gc
#データの読み込み
def getExperimentalData(file):
    path = os.getcwd()+"/../data_ml/" #データがあるフォルダの指定
    data = pd.read_excel(path + file, sheet_name='result') #データの読み込み
    data_log = data.copy()
    return data, data_log

#欠損値を除いたデータセットの作成
def getDataset(data, param):
    df = data.dropna(subset=[param]) # "param"列に欠損値がある行を削除
    return df

#出力ディレクトリの設定
def makeTdDir():
    folder_path = os.getcwd()+ "/../graph" #グラフを保存するフォルダのパスを指定
    if not os.path.isdir(folder_path): #指定したフォルダがない時
        os.mkdir(folder_path) #指定したフォルダを作成
    td = dt.today() #今日の日付の設定
    tdstr = td.strftime('%y%m%d') #今日の日付を6桁の文字列で受け取る
    td_path = folder_path + "/graph" + tdstr #今日作成したグラフを保存するフォルダのパスを指定
    if not os.path.isdir(td_path):
        os.mkdir(td_path)
    return tdstr, td_path + "/" 

def makeDir(param):
    if not os.path.isdir(td_path + param):
        os.mkdir(td_path + param) #出力パラメータごとのファイルを作成
    return td_path + param+"/"

tdstr,td_path = makeTdDir()

In [1]:
from sklearn.ensemble import StackingRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import cross_val_score


param_grid_svr={"C":[2**-5,2**-3,2**-1,2**1,2**3,2**5,2**7,2**9,2**11],
                "gamma":[2**-20,2**-18,2**-16,2**-14,2**-12,2**-10,2**-8,2**-6,2**-4,2**-2,2**0,2**2,2**4,2**6,2**8,2**10],
                "kernel":["rbf"],
                  "epsilon":[2**-10,2**-8,2**-6,2**-4,2**-2,2**0]}

param_grid_gbdt = {"learning_rate":[i/100 for i in range(1,51,1)],  
                   "max_depth":[2,3,4,5],
                   "n_estimators":[300]}

param_grid_knn={"n_neighbors":[1,2,3,4]}


def NN_model():
    
    model = Sequential()
    
    model.add(Dense(16, activation='relu', input_dim=len(variable)))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1, activation='relu'))
    
    model.compile(optimizer=Adam(learning_rate=0.001),
              loss='mean_squared_error',
              metrics=['accuracy'])
    
    return model

ModuleNotFoundError: No module named 'scikeras'

In [ ]:
#個体の各遺伝子を決めるために使用
import random
#DEAPの中にある必要なモジュールをインポート
from deap import base
from deap import creator
from deap import tools
from deap import algorithms

# Define a custom mutation function for floats
def mutate_float(individual, low, up, indpb):
    for i, val in enumerate(individual):
        if random.random() < indpb:
            individual[i] = random.uniform(low, up)
    return individual,

In [ ]:
data=pd.read_excel("")
data

In [ ]:
variable=data.columns[4:]
variable
param="Max Rate (1/h)"

In [ ]:
standard = StandardScaler()
minmax = MinMaxScaler()

data_scaled=data[variable].copy()
data_scaled[data_scaled.columns] = standard.fit_transform(data_scaled)
data_scaled[data_scaled.columns] = minmax.fit_transform(data_scaled)
data_scaled[param]=data[param]
    
mlp = KerasRegressor(model=NN_model, batch_size=16, epochs=100,verbose=0)
gbdt=GridSearchCV(GradientBoostingRegressor(),param_grid_gbdt,cv=5)
svr=GridSearchCV(SVR(),param_grid_svr,cv=5)
knn=GridSearchCV(KNeighborsRegressor(),param_grid_knn,cv=5)
    
gbdt.fit(data_scaled[variable] , data_scaled[param])
svr.fit(data_scaled[variable] , data_scaled[param])
knn.fit(data_scaled[variable] , data_scaled[param])


final=pd.DataFrame()
num=1

while num<11:
    
    data_scaled=data[variable].copy()
    data_scaled[data_scaled.columns] = standard.transform(data_scaled)
    data_scaled[data_scaled.columns] = minmax.transform(data_scaled)
    data_scaled["Max Rate (1/h)"]=data["Max Rate (1/h)"]

    gbdt_en=GradientBoostingRegressor(**gbdt.best_params_)
    svr_en=SVR(**svr.best_params_)
    knn_en=KNeighborsRegressor(**knn.best_params_)
    estimators = [
            ('svr', svr_en),
            ("gbdt",gbdt_en),
            ('mlp', mlp),
            ('knn', knn_en)]
    
    model= StackingRegressor(estimators=estimators,final_estimator=LinearRegression())
    model.fit(data_scaled[variable] , data_scaled[param])

    #最大化問題として設定
    creator.create( "FitnessMax", base.Fitness, weights=(1.0,) )
    #個体の定義（list型と指定しただけで、中身の遺伝子は後で入れる）
    creator.create("Individual", list, fitness = creator.FitnessMax )

    # 目的関数の定義

    def evaluate(individual):

        arti_data=pd.DataFrame(individual).T
        arti_data.columns=variable

        prediction = model.predict(arti_data)
        return prediction,

    #各種関数の設定を行います
    toolbox = base.Toolbox()
    #random.uniformの別名をattribute関数として設定。各個体の遺伝子の中身を決める関数(各遺伝子は0～10のランダムな整数)
    toolbox.register("attribute", random.uniform, 0.01,1.0)
    #individualという関数を設定。それぞれの個体に含まれる6個の遺伝子をattributeにより決めるよ、ということ。
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attribute, len(variable))
    #集団の個体数を設定するための関数を準備
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    #トーナメント方式で次世代に子を残す親を選択（tornsizeは各トーナメントに参加する個体の数）
    toolbox.register("select", tools.selTournament, tournsize=5)
    #交叉関数の設定。一点交叉を採用
    toolbox.register("mate", tools.cxOnePoint)
    #突然変異関数の設定。indpbは各遺伝子が突然変異を起こす確率。変異は0~20の整数で変異
    toolbox.register("mutate", mutate_float, low=0.2, up=2.0, indpb=0.2)
    #評価したい関数の設定（目的関数のこと）
    toolbox.register("evaluate", evaluate)

    #乱数の固定
    random.seed(128)
    #何世代まで行うか
    NGEN = 50
    #集団の個体数
    POP = 300
    #交叉確率
    CXPB = 0.9
    #個体が突然変異を起こす確率
    MUTPB = 0.2
    #集団は80個体という情報の設定
    pop = toolbox.population(n=POP)

    #集団内の個体それぞれの適応度（目的関数の値）を計算
    for ind in pop:
        ind.fitness.values = toolbox.evaluate(ind)

    #パレート曲線上の個体(つまり、良い結果の個体)をhofという変数に格納
    hof = tools.ParetoFront()

    #最も単純なSimple GAという進化戦略を採用
    algorithms.eaSimple(pop, toolbox, cxpb=CXPB, mutpb=MUTPB, ngen=NGEN, halloffame=hof)

    #最終的な集団(pop)からベストな個体を1体選出する関数
    best_ind = tools.selBest(pop, 1)[0]

    artificial=pd.DataFrame(best_ind).T
    artificial.columns=variable
    
    #スケーリング
    artificial[artificial.columns] = minmax.inverse_transform(artificial)
    artificial[artificial.columns] = standard.inverse_transform(artificial)  # scalerはモデル構築時に使用したスケーラー
    
        
    artificial["predicted"]=best_ind.fitness.values[0][0]
    
    final=pd.concat([final, artificial])
    
    artificial["Max Rate (1/h)"]=0
    data=pd.concat([data, artificial])
    
    num+=1
    gc.collect()

In [ ]:
final

In [ ]:
path=makeDir("prediction_result")
final.to_csv(path + tdstr + "_media_prediction_result.csv")